In [ ]:
from transformers import CLIPProcessor, CLIPModel, CLIPVisionModelWithProjection

model = CLIPVisionModelWithProjection.from_pretrained(
    pretrained_model_name_or_path='openai/clip-vit-large-patch14',
    ignore_mismatched_sizes=True
)